In [1]:
import os
import pymupdf
import cv2
import numpy as np

## Get faces from all pages from the yearbooks

In [20]:
detec = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")

for file in os.listdir("./yearbooks"):
    pdf_path = os.path.join("./yearbooks", file)
    print("Getting faces from", pdf_path)
    # Create path to store the image of each face
    year = file.split(".")[0]
    images_path = f"./images/{year}/"
    if not os.path.isdir(images_path):
        os.makedirs(images_path)

    pdf = pymupdf.open(pdf_path)

    for page in pdf:
        pix = page.get_pixmap()
        img =  np.frombuffer(buffer=pix.samples, dtype=np.uint8).reshape((pix.height, pix.width, 3))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        face = detec.detectMultiScale(gray, 1.3, 3)

        for i, (x, y, w, h) in enumerate(face):
            crop = img[y:y+h,x:x+w]
            cv2.imwrite(images_path + f"{page.number}_{i}.png", crop)
        
            break
        
        if not isinstance(face, tuple):
            break


    


Getting faces from ./yearbooks/1951.pdf
Getting faces from ./yearbooks/1931.pdf
Getting faces from ./yearbooks/1971.pdf
Getting faces from ./yearbooks/1961.pdf
Getting faces from ./yearbooks/1911.pdf
Getting faces from ./yearbooks/1921.pdf
Getting faces from ./yearbooks/1941.pdf
